
# Import Packages

In [2]:
import pandas as pd 
import math
import regex
import re
import numpy as np
from numpy import random
from sklearn.impute import SimpleImputer

import os
from datetime import date
from datetime import datetime
from datetime import timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Import all datasets

In [3]:
# hdb resale price
df_resaleprice = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

# region mapping file
df_region = pd.read_excel('region.xlsx')

In [7]:
#number of rows, columns, and firstr 2 rows
print(df_resaleprice.shape)
df_resaleprice.head(2)

(147534, 11)


,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0


In [12]:
df_resaleprice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147534 entries, 0 to 147533
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                147534 non-null  object 
 1   town                 147534 non-null  object 
 2   flat_type            147534 non-null  object 
 3   block                147534 non-null  object 
 4   street_name          147534 non-null  object 
 5   storey_range         147534 non-null  object 
 6   floor_area_sqm       147534 non-null  float64
 7   flat_model           147534 non-null  object 
 8   lease_commence_date  147534 non-null  int64  
 9   remaining_lease      147534 non-null  object 
 10  resale_price         147534 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 12.4+ MB


# Data Preparation

## Derive YEAR (YYYY) & MONTH (MM) from month column (YYYY-MM)

In [19]:
def derive_year_mth(df, col_name):
    year = df[col_name].str[:4] # Extract the first four chars from column
    month = df[col_name].str[-2:] # Extract the last two chars from column
    # Convert the year & month values to int
    year = pd.to_numeric(year, errors="coerce")
    month = pd.to_numeric(month, errors="coerce")
    df["YEAR"] = year
    df["MONTH"] = month

derive_year_mth(df_resaleprice, "month")

In [20]:
#Rename original "month" (2017-01) column name to "MY" to avoid confusion

df_resaleprice = df_resaleprice.rename(columns={"month": "MY"})

## Extract remaining lease year (YY) and months(MM) from remaining lease column (YY years MM months)

In [ ]:
def derive_lease_year_month(df, col_name):
    pattern1 = r'(\d+)\s*(?:years?|y)\s*(\d+)\s*(?:months?|m)?'
    pattern2 = r'(\d+)\s*(year|years|yrs|yr)?\s*(\d+)?\s*(month|months|mons|mon)?'
    month = df[col_name].str.extract(pattern1)
    year = df[col_name].str.extract(pattern2)
    df["REMAINING_LEASE_YEAR"] = pd.to_numeric(year[0])
    df["REMAINING_LEASE_MONTH"] = pd.to_numeric(month[1])
    return df

derive_lease_year_month(df_resaleprice, "remaining_lease")

In [23]:
df_resaleprice['REMAINING_LEASE_MONTH'] = df_resaleprice['REMAINING_LEASE_MONTH'].fillna(0)

## Calculate total number of remaining leasing months

In [39]:
df_resaleprice['REMAINING_LEASE_MONTHS'] = (df_resaleprice['REMAINING_LEASE_YEAR']*12) + df_resaleprice['REMAINING_LEASE_MONTH']

## Update town column to uppercase
- To handle case mismatch in df_region and df_resaleprice

In [25]:
df_region = df_region.rename(columns={'Town Centres / Areas': 'town'})
df_region['town'] = df_region['town'].str.upper()

In [27]:
df_region['town'].unique()

array(['ADMIRALITY', 'HOLLAND', 'SERANGOON', 'LIM CHU KANG', 'THOMSON',
       'KRANJI', 'QUEENSTOWN', 'PUNGGOL', 'CHOA CHU KANG', 'MARYMOUNT',
       'WOODLANDS', 'BUKIT MERAH', 'HOUGANG', 'BUKIT PANJANG', 'SIN MING',
       'SEMBAWANG', 'TELOK BLANGAH', 'TAMPINES', 'TUAS', 'ANG MO KIO',
       'YISHUN', 'PASIR PANJANG', 'PASIR RIS', 'JURONG EAST', 'BISHAN',
       'YIO CHU KANG', 'SENTOSA', 'LOYANG', 'JURONG WEST',
       'SERANGOON GARDENS', 'SELETAR', 'BUKIT TIMAH', 'SIMEI',
       'JURONG INDUSTRIAL ESTATE', 'MACRITCHIE', 'SENGKANG', 'NEWTON',
       'KALLANG', 'BUKIT BATOK', 'TOA PAYOH', 'ORCHARD', 'KATONG',
       'HILLVIEW', 'CITY', 'EAST COAST', 'WEST COAST', 'MARINA SOUTH',
       'MACPHERSON', 'CLEMENTI', 'BEDOK', 'PULAU UBIN', 'PULAU TEKONG',
       'CENTRAL AREA', 'GEYLANG', 'KALLANG/WHAMPOA', 'MARINE PARADE'],
      dtype=object)

In [29]:
df_resaleprice['town'] = df_resaleprice['town'].str.upper()

## Concatenate block and street name to get ADDRESS
- Address is later used as key to merge df_resaleprice and df_houseinfo

In [34]:
df_resaleprice["ADDRESS"] = df_resaleprice["block"].astype(str) + " " + df_resaleprice["street_name"]

## Replace outliers value with median

In [ ]:
upper_bound = df_resaleprice["resale_price"].mean() + 3 * df_resaleprice["resale_price"].std()

df_resaleprice["resale_price"] = np.where(df_resaleprice["resale_price"] > upper_bound, "", df_resaleprice["resale_price"])

imp_median = SimpleImputer(strategy='median')
df_resaleprice["resale_price"] = imp_median.fit_transform(df_resaleprice["resale_price"])

## Export to Excel

In [ ]:
df_region.to_excel(r'region.xlsx', index=False)
df_resaleprice.to_excel(r'resale.xlsx', index=False)